# Machine learning algorithms for coral bleaching classification 

## Load dataset

In [1]:
'''
    Import libraries
'''
from sklearn import datasets
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import sklearn
import seaborn as sb
import matplotlib.pyplot as plt
from numpy import mean 
from numpy import std
from scipy import stats
import scipy.stats as scipystats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats.stats import linregress
from scipy.stats.stats import pearsonr
# check scikit-learn version
print(sklearn.__version__)

0.22.2.post1


In [2]:
'''
    Load full dataset
'''
data = pd.read_csv('df_sst_clouds.csv')
len(data)  

21003

In [3]:
'''
    Subset DF by SEVERITY_CODE [0,1,2,3]
'''
#data = data.dropna() # drop rows that contains NaN's 
data = data[(data.SEVERITY_CODE == 0)|(data.SEVERITY_CODE == 1)|(data.SEVERITY_CODE == 2)|(data.SEVERITY_CODE == 3)] 
#data = data[(data.YEAR >= 2005)] # First year with more than 100 records
#list(data.columns)
data = data.dropna() # drop rows that contains NaN's
len(data)

20263

In [7]:
data.columns


Index(['ITEM_ID', 'COUNTRY', 'lat', 'lon', 'SEVERITY_CODE', 'DHW_class', 'DAY',
       'MONTH', 'YEAR', 'full_date', 'JD', 'SST', 'SST_adj_date',
       'SSTrunmax30', 'SSTrunmax30_adj_date', 'SSTrunmean30',
       'SSTrunmean30_adj_date', 'SSTrunmean7', 'SSTrunmean7_adj_date',
       'SSTrunmax7', 'SSTrunmax7_adj_date', 'SSTrunmean90',
       'SSTrunmean90_adj_date', 'SSTrunmax90', 'SSTrunmax90_adj_date', 'DHW',
       'DHW_adj_date', 'DHWrunmean7', 'DHWrunmean7_adj_date', 'DHWrunmax7',
       'DHWrunmax7_adj_date', 'DHWrunmean30', 'DHWrunmean30_adj_date',
       'DHWrunmax30', 'DHWrunmax30_adj_date', 'DHWrunmean90',
       'DHWrunmean90_adj_date', 'DHWrunmax90', 'DHWrunmax90_adj_date', 'DHW_9',
       'DHW_9_adj_date', 'CF', 'CF_adj_date', 'CFrunmean30',
       'CFrunmean30_adj_date', 'CFrunmean7', 'CFrunmean7_adj_date',
       'CFrunmean90', 'CFrunmean90_adj_date', 'CF_monmean',
       'CF_monmean_adj_date', 'CF_a', 'CF_a_adj_date', 'CF_a_runmean30',
       'CF_a_runmean30_adj_date'

# Calculate a linear least-squares regression for two sets of measurements

In [5]:
reglinSEVERITY_CODE_WD = linregress(data.SEVERITY_CODE, data.WD)
print('SEVERITY_CODE vs WD:', reglinSEVERITY_CODE_WD)
print('')
reglinSEVERITY_CODE_DHW = linregress(data.SEVERITY_CODE, data.DHW)
print('SEVERITY_CODE vs DHW:', reglinSEVERITY_CODE_DHW)
print('')
reglinSEVERITY_CODE_DHW_9 = linregress(data.SEVERITY_CODE, data.DHW_9)
print('SEVERITY_CODE vs DHW_9:', reglinSEVERITY_CODE_DHW_9)


SEVERITY_CODE vs WD: LinregressResult(slope=12.15953209322459, intercept=7.7975052177288475, rvalue=0.39223935815290834, pvalue=0.0, stderr=0.20033576786692894)

SEVERITY_CODE vs DHW: LinregressResult(slope=1.3235402529731053, intercept=0.3375873072556206, rvalue=0.5174390045937328, pvalue=0.0, stderr=0.015377271504354048)

SEVERITY_CODE vs DHW_9: LinregressResult(slope=1.3272295295423502, intercept=1.029564380233199, rvalue=0.5427642388488548, pvalue=0.0, stderr=0.014428601822049726)


In [6]:
#To get coefficient of determination (R-squared) do rvalue**2:
reglinSEVERITY_CODE_DHW.rvalue**2

0.26774312347495305

In [36]:
reg = smf.ols(formula = "DHW ~ SEVERITY_CODE", data = data).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DHW   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     7408.
Date:                Mon, 23 Nov 2020   Prob (F-statistic):               0.00
Time:                        09:00:16   Log-Likelihood:                -46390.
No. Observations:               20263   AIC:                         9.278e+04
Df Residuals:                   20261   BIC:                         9.280e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3376      0.025     13.700      0.000       0.289       0.386
SEVERITY_CODE     1.3235      0.015     86.071      0.000       1.293       1.354
==============================================================================
Omnibus:                     5563.249   Durbin-Watson:                   0.632
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20879.238
Skew:                           1.338   Prob(JB):                         0.00
Kurtosis:                       7.191   Cond. No.                         2.93
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
from sklearn import linear_model
X = data[['DHW_9','CF_a_runmean90']] 
y = data['SEVERITY_CODE']
regr = linear_model.LinearRegression()
regr.fit(X, y)
print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)
# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

Intercept: 
 0.5986177316293095
Coefficients: 
 [0.22247007 0.05913042]
                            OLS Regression Results                            
Dep. Variable:          SEVERITY_CODE   R-squared:                       0.295
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     4232.
Date:                Mon, 23 Nov 2020   Prob (F-statistic):               0.00
Time:                        12:56:46   Log-Likelihood:                -26980.
No. Observations:               20263   AIC:                         5.397e+04
Df Residuals:                   20260   BIC:                         5.399e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------

# Regplot

In [ ]:
Cf_a_DHW = data[['CF_a_runmean90','DHW','SEVERITY_CODE']]
scatter = sb.regplot(Cf_a_DHW.SEVERITY_CODE, Cf_a_DHW.DHW, marker='+',scatter_kws={'s':1}, line_kws={'lw':1, 'color':'darkred'}) 
#plt.savefig('regplotDHW_CF_a_90.pdf', dpi=300)

# Pairplots

In [ ]:
 
# Create individial series per variable and severity 
Cf_DHW = data[['CF_a_runmean90','DHW','SEVERITY_CODE']]
pairgrid = sb.pairplot(Cf_DHW, hue='SEVERITY_CODE', palette="hls", kind='reg', markers='.', corner=True, plot_kws={'scatter_kws':{'alpha': 0.5,'edgecolor': 'none'}},size = 10)
plt.show()
#plt.savefig('pairplotDHW_CF_a_90.pdf', dpi=300)